### Establishing a connection with Python

Next, let's create a Python notebook in our folder named `pg_connect.ipynb`. This will be where we do our work.

Now we need to connect to a Python environment. For this course, I had T&I establish a pre-configured environment called `DAT153-shared`, which you should be able to access by clicking the "Select Kernel" button in the upper-right corner of the notebook.

Once we're connected to the right kernel, let's add a new code cell at the top and import the packages we need:

- `sqlalchemy` will help us connect to Postgres
- `pandas` will help us store and manipulate the data in data frames
- `os` gives us tools to store passwords and other secrets like API keys
- `dotenv` lets us read key-value pairs from the `.env` file we created earlier

Then we can run `load_dotenv()` to retrieve our password from the `.env` file:

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In the next code cell, we can establish the connection parameters for PostgreSQL:

In [ ]:
username = 'pebenbow'
password = os.getenv('PG_PASSWORD')
host = 'dat153db.ada.davidson.edu'
port = '63560'
database = 'houston'

connection_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"

engine = create_engine(connection_url)

The `create_engine()` function returns an "engine" object, which is SQLAlchemy's way of creating and maintaining database connections.

Now we can get to the fun stuff!

### Querying the database with Python

The `inspect()` function in SQLAlchemy allows us to retrieve metadata about our database, including tables, columns, keys, indexes, and so forth. This command loops through all the tables of a given schema and returns these features of the database:

In [ ]:
from sqlalchemy import inspect

inspector = inspect(engine)

schema_name = 'solution1'

# Get a list of all table names
table_names = inspector.get_table_names(schema_name)
print("Tables:", table_names)

# Iterate through tables and print column information
for table_name in table_names:
    print(f"\nTable: {table_name}")
    columns = inspector.get_columns(table_name,schema_name)
    for col in columns:
        print(f"  Column: {col['name']} (Type: {col['type']}, Nullable: {col['nullable']})")

    # You can also get other metadata like primary keys, foreign keys, and indexes
    primary_keys = inspector.get_pk_constraint(table_name,schema_name)
    print(f"  Primary Keys: {primary_keys.get('constrained_columns')}")

    foreign_keys = inspector.get_foreign_keys(table_name,schema_name)
    print(f"  Foreign Keys: {foreign_keys}")

    indexes = inspector.get_indexes(table_name,schema_name)
    print(f"  Indexes: {indexes}")

Next, we can run some `SELECT` statements:

In [ ]:
query = "SELECT * FROM solution1.funds;"

df = pd.read_sql(query, engine)

print(df)